In [1]:
import math
import pickle
import random
import sys
from multiprocessing import Pool

import networkx as nx
import numpy as np
import pandas as pd
from scipy.spatial import KDTree as kd
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
import graph_tests as city_tests
import graph_generator

import osmnx as ox


In [2]:


def get_graph(city_id: str = 'R2555133') -> nx.Graph:
    gdf = ox.geocode_to_gdf(city_id, by_osmid=True)
    polygon_boundary = gdf.unary_union
    graph = ox.graph_from_polygon(polygon_boundary,
                                  network_type='drive',
                                  simplify=True)
    G = nx.Graph(graph)
    H = nx.Graph()
    # Добавляем рёбра в новый граф, копируя только веса
    for u, d in G.nodes(data=True):
        H.add_node(u, x=d['x'], y=d['y'])
    for u, v, d in G.edges(data=True):
        H.add_edge(u, v, length=d['length'])
    del city_id, gdf, polygon_boundary, graph, G
    return H


def calculate(data):
    cities = data[0]
    points_number = data[1]
    NUMBER = data[2]
    THREADS = data[3]

    for name, id in cities:
        G = get_graph(id)

        points = [graph_generator.get_node_for_initial_graph_v2(G) for _ in
                  range(points_number)]

        Q = G.copy()
        for u in Q.nodes:
            if u in Q[u]:
                Q.remove_edge(u, u)
        data = {}
        for (u, d) in Q.degree:
            if d not in data:
                data[d] = 0
            data[d] += 1
        dd = {
            'degree': [],
            'probability': []
        }
        total = sum([d for d in data.values()])

        for d in data:
            dd['degree'].append(d)
            dd['probability'].append(data[d] / total)
        df = pd.DataFrame.from_dict(dd)
        print(df)
        return df

In [3]:


if __name__ == '__main__':
    total = 1
    points_number = 500
    if len(sys.argv) == 2:
        total = int(sys.argv[1])

    # print('THREADS:', total)
    # print('POINTS:', points_number)

    cities = {
        # 'ASHA': 'R13470549',
        # 'KRG': 'R4676636',
        # 'EKB': 'R6564910',
        # 'BARCELONA': 'R347950',
        'PARIS': 'R71525',
        # 'Prague': 'R435514',
        # 'MSK': 'R2555133',
        # 'SBP': 'R337422',
        # 'SINGAPORE': 'R17140517',
        # 'BERLIN': 'R62422',
        # 'ROME': 'R41485',
        # 'LA': 'R207359',
        # 'DUBAI': 'R4479752',
        # 'RIO': 'R2697338',
        # 'DELHI': 'R1942586',
        # 'KAIR': 'R5466227'
    }
    total_len = len(cities)
    l = list(cities.items())
    data = [[l[i: total_len: total], points_number, i + 1, total] for i in range(total)]

    df = calculate(data[0])

/tmp/ipykernel_4064/168065684.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  polygon_boundary = gdf.unary_union


   degree  probability
0       4     0.230931
1       3     0.654340
2       2     0.040139
3       5     0.016435
4       1     0.056679
5       6     0.001370
6       7     0.000105


In [4]:
df

,degree,probability
0,4,0.230931
1,3,0.654340
2,2,0.040139
3,5,0.016435
4,1,0.056679
5,6,0.001370
6,7,0.000105


In [5]:
momentums = {}
for i in range(1, 10):
    momentums[i] = 0
    for j in range(len(df)):
        momentums[i] += (df['degree'][j] ** i) * df['probability'][j]


In [6]:
for m in momentums:
    print(m, ':', momentums[m])

1 : 3.114833544037084
2 : 10.266540244416351
3 : 35.21091445427728
4 : 125.11862621154657
5 : 460.59903076274753
6 : 1758.6230509903075
7 : 6973.946691951117
8 : 28765.34618626212
9 : 123590.13126843657


In [7]:
degree_destrib = {
    0: 0,
    3: 0.816342,
    2: 0.044013,
    1: 0.071698,
    4: 0.061802,
    5: 0.005618,
    7: 0.000017
}

q = list(degree_destrib.items())
q.sort(key=lambda x: x[0])
degree_destrib = dict(q)
W = list(degree_destrib.values())

W = np.array(W)
W/=np.sum(W)
print(W)


[0.00000000e+00 7.17345846e-02 4.40354581e-02 8.16758547e-01
 6.18335351e-02 5.62086664e-03 1.70086744e-05]


In [8]:
S = nx.expected_degree_graph(W)

In [9]:
# print(len(S.nodes))

In [10]:
print(sum(W))

1.0


In [11]:
print(W)

[0.00000000e+00 7.17345846e-02 4.40354581e-02 8.16758547e-01
 6.18335351e-02 5.62086664e-03 1.70086744e-05]


In [12]:
# print(c)

In [13]:
N = 100
degrees = np.random.choice(range(0, len(W)), size=N,p=W)
print(degrees)

[3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 1 3 5 3 3 4 3 3 3 1 3 3 3 3 3 3 4 3 3 3 3 3
 3 2 3 3 3 3 3 3 1 3 3 3 4 3 3 3 3 3 4 3 3 3 4 3 2 3 3 3 3 3 3 3 3 1 3 4 3
 2 3 3 3 3 3 3 3 3 4 3 3 3 3 2 3 3 4 3 3 2 2 3 3 3 3]


In [14]:
degree_destrib = {
        0: 0,
        3: 0.816342,
        2: 0.044013,
        1: 0.071698,
        4: 0.061802,
        5: 0.005618,
        7: 0.000017
    }

q = list(degree_destrib.items())
q.sort(key=lambda x: x[0])
degree_destrib = dict(q)
W = list(degree_destrib.values())

W = np.array(W)
W/=np.sum(W)
print(W)

[0.00000000e+00 7.17345846e-02 4.40354581e-02 8.16758547e-01
 6.18335351e-02 5.62086664e-03 1.70086744e-05]


In [15]:
N = 100
degrees = np.random.choice(range(0, len(W)), size=N,p=W)

In [16]:
degrees

array([3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3,
       2, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 4,
       3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3,
       3, 4, 3, 1, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 4,
       3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3])

In [17]:
def handshake(nodes):
    total = 0
    for n in nodes:
        if n % 2 == 1:
            total +=1
    return total%2==0
N = 10000
degrees = np.random.choice(range(0, len(W)), size=N,p=W)
print(handshake(degrees))

True


In [18]:
def havel_hakimi(seq):
    # if seq is empty or only contains zeros,
    # degree sequence is valid
    if len(seq) < 1 or all(deg == 0 for deg in seq):
        print("Finished! Graph IS constructable with Havel Hakimi algorithm.")
        return True

    print(seq, end="")
    seq.sort()
    print(" --sort--> ", end="")
    print(seq)

    last = seq[len(seq)-1]
    if last > len(seq)-1:
        print("Failed! Graph IS NOT constructable with Havel Hakimi algorithm.")
        return False

    print(seq, end="")

    # remove last element
    seq.remove(last)

    # iterate seq backwards
    for num in range(len(seq)-1, len(seq)-last-1, -1):
        if seq[num] > 0:
            seq[num] -= 1
        else:
            print("\nFailed! Graph is not constructable with Havel Hakimi algorithm")
            return False

    print(" --alg-->", end="")
    print(seq)

    # recursive call
    return havel_hakimi(seq)

In [19]:
data

[[[('PARIS', 'R71525')], 500, 1, 1]]

In [20]:
G.edges(data=True)

NameError: name 'G' is not defined

In [ ]:
N = len(G.nodes)
M = len(G.edges)
print(M/N)

In [ ]:
def f(d):
    G = nx.random_geometric_graph(10000, 0.1)

In [ ]:
d = [i for i in range(10000)]
with Pool(20) as p:
    r = list(tqdm(p.imap_unordered(f, d), total = len(d)))